In [65]:
import requests
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime
from io import StringIO

from airflow.decorators import dag, task
from airflow.operators.python import get_current_context
from airflow.models import Variable

In [66]:
myYear = 1994 + hash(f'v-kachanovskij-24') % 23
myYear

2002

In [67]:
PATH = '/var/lib/airflow/airflow.git/dags/v-kachanovskij-24/vgsales.csv'
login = 'v-kachanovskij-24'
myYear = 1994 + hash(f'{login}') % 23
myYear

2002

In [68]:
#data = pd.read_csv(PATH)
#data = data.query('Year == @year')
#data

In [69]:
vg_df = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-kachanovskij-24/airflow_homework/vgsales.csv')
vg_my_df = vg_df.query('Year == @myYear')

In [70]:
vg_my_df

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
24,25,Grand Theft Auto: Vice City,PS2,2002.0,Action,Take-Two Interactive,8.41,5.49,0.47,1.78,16.15
25,26,Pokemon Ruby/Pokemon Sapphire,GBA,2002.0,Role-Playing,Nintendo,6.06,3.90,5.38,0.50,15.85
114,115,Medal of Honor: Frontline,PS2,2002.0,Shooter,Electronic Arts,2.93,2.75,0.17,0.99,6.83
131,132,Kingdom Hearts,PS2,2002.0,Role-Playing,Sony Computer Entertainment,3.64,1.20,1.49,0.07,6.40
135,136,Super Mario Sunshine,GC,2002.0,Platform,Nintendo,4.01,1.26,0.87,0.17,6.31
...,...,...,...,...,...,...,...,...,...,...,...
16426,16429,ESPN NBA 2Night 2002,XB,2002.0,Sports,Konami Digital Entertainment,0.01,0.00,0.00,0.00,0.01
16464,16467,Egg Mania: Eggstreme Madness,GC,2002.0,Puzzle,Kemco,0.01,0.00,0.00,0.00,0.01
16556,16559,Super Robot Taisen: Original Generation,GBA,2002.0,Role-Playing,Banpresto,0.01,0.00,0.00,0.00,0.01
16563,16566,Mortal Kombat: Deadly Alliance,GBA,2002.0,Fighting,Midway Games,0.01,0.00,0.00,0.00,0.01


In [71]:
vg_df

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [72]:
default_args = {
    'owner': 'v-kachanovskij-24',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2022, 10, 27),
    'schedule_interval': '0 12 * * *'
}

CHAT_ID = -620798068
try:
    BOT_TOKEN = Variable.get('telegram_secret')
except:
    BOT_TOKEN = ''

def send_message(context):
    date = context['ds']
    dag_id = context['dag'].dag_id
    message = f'Huge success! Dag {dag_id} completed on {date}'
    if BOT_TOKEN != '':
        bot = telegram.Bot(token=BOT_TOKEN)
        bot.send_message(chat_id=CHAT_ID, text=message)
    else:
        pass

@dag(default_args=default_args, catchup=False)
def top_10_airflow_2():
    @task(retries=3)
    def get_data():
        top_doms = requests.get(TOP_1M_DOMAINS, stream=True)
        zipfile = ZipFile(BytesIO(top_doms.content))
        top_data = zipfile.read(TOP_1M_DOMAINS_FILE).decode('utf-8')
        return top_data

    @task(retries=4, retry_delay=timedelta(10))
    def get_table_ru(top_data):
        top_data_df = pd.read_csv(StringIO(top_data), names=['rank', 'domain'])
        top_data_ru = top_data_df[top_data_df['domain'].str.endswith('.ru')]
        return top_data_ru.to_csv(index=False)

    @task()
    def get_stat_ru(top_data_ru):
        ru_df = pd.read_csv(StringIO(top_data_ru))
        ru_avg = int(ru_df['rank'].aggregate(np.mean))
        ru_median = int(ru_df['rank'].aggregate(np.median))
        return {'ru_avg': ru_avg, 'ru_median': ru_median}

    @task()
    def get_table_com(top_data):
        top_data_df = pd.read_csv(StringIO(top_data), names=['rank', 'domain'])
        top_data_com = top_data_df[top_data_df['domain'].str.endswith('.com')]
        return top_data_com.to_csv(index=False)

    @task()
    def get_stat_com(top_data_com):
        com_df = pd.read_csv(StringIO(top_data_com))
        com_avg = int(com_df['rank'].aggregate(np.mean))
        com_median = int(com_df['rank'].aggregate(np.median))
        return {'com_avg': com_avg, 'com_median': com_median}

    @task(on_success_callback=send_message)
    def print_data(ru_stat, com_stat):

        context = get_current_context()
        date = context['ds']

        ru_avg, ru_median = ru_stat['ru_avg'], ru_stat['ru_median']
        com_avg, com_median = com_stat['com_avg'], com_stat['com_median']

        print(f'''Data from .RU for {date}
                  Avg rank: {ru_avg}
                  Median rank: {ru_median}''')

        print(f'''Data from .COM for {date}
                          Avg rank: {com_avg}
                          Median rank: {com_median}''')

    top_data = get_data()
    top_data_ru = get_table_ru(top_data)
    ru_data = get_stat_ru(top_data_ru)

    top_data_com = get_table_com(top_data)
    com_data = get_stat_com(top_data_com)

    print_data(ru_data, com_data)

top_10_airflow_2 = top_10_airflow_2()


Дальше нужно составить DAG из нескольких тасок, в результате которого нужно будет найти ответы на следующие вопросы:

#### - Какая игра была самой продаваемой в этом году во всем мире?

#### - Игры какого жанра были самыми продаваемыми в Европе? Перечислить все, если их несколько

#### - На какой платформе было больше всего игр, которые продались более чем миллионным тиражом в Северной Америке? Перечислить все, если их несколько

#### - У какого издателя самые высокие средние продажи в Японии? Перечислить все, если их несколько

#### - Сколько игр продались лучше в Европе, чем в Японии?

In [73]:
vg_my_df

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
24,25,Grand Theft Auto: Vice City,PS2,2002.0,Action,Take-Two Interactive,8.41,5.49,0.47,1.78,16.15
25,26,Pokemon Ruby/Pokemon Sapphire,GBA,2002.0,Role-Playing,Nintendo,6.06,3.90,5.38,0.50,15.85
114,115,Medal of Honor: Frontline,PS2,2002.0,Shooter,Electronic Arts,2.93,2.75,0.17,0.99,6.83
131,132,Kingdom Hearts,PS2,2002.0,Role-Playing,Sony Computer Entertainment,3.64,1.20,1.49,0.07,6.40
135,136,Super Mario Sunshine,GC,2002.0,Platform,Nintendo,4.01,1.26,0.87,0.17,6.31
...,...,...,...,...,...,...,...,...,...,...,...
16426,16429,ESPN NBA 2Night 2002,XB,2002.0,Sports,Konami Digital Entertainment,0.01,0.00,0.00,0.00,0.01
16464,16467,Egg Mania: Eggstreme Madness,GC,2002.0,Puzzle,Kemco,0.01,0.00,0.00,0.00,0.01
16556,16559,Super Robot Taisen: Original Generation,GBA,2002.0,Role-Playing,Banpresto,0.01,0.00,0.00,0.00,0.01
16563,16566,Mortal Kombat: Deadly Alliance,GBA,2002.0,Fighting,Midway Games,0.01,0.00,0.00,0.00,0.01


----------------------------------------------

##### - Какая игра была самой продаваемой в этом году во всем мире?


In [105]:
max_sales_global = vg_my_df.groupby('Name',as_index = False) \
    .agg({'Global_Sales' : 'sum'}) \
    .sort_values('Global_Sales', ascending = False) \
    .iloc[0]['Name']


--------------

##### - Игры какого жанра были самыми продаваемыми в Европе? Перечислить все, если их несколько


In [129]:
eu_mp_sales = vg_my_df.groupby('Genre',as_index = False) \
    .agg({'EU_Sales' : 'sum'}) \
    .sort_values('EU_Sales', ascending = False) \
    .head(5)
eu_mp_sales_top1 = eu_mp_sales.iloc[0]['Genre']

------

##### - На какой платформе было больше всего игр, которые продались более чем миллионным тиражом в Северной Америке? Перечислить все, если их несколько

In [103]:
na_mp_platform = vg_my_df.query('NA_Sales > 1') \
    .groupby('Platform', as_index = False) \
    .agg({'Name' : 'count'}) \
    .sort_values('Name', ascending = False)
na_mp_platform_top1 = na_mp_platform.iloc[0]['Platform']

----

##### - У какого издателя самые высокие средние продажи в Японии? Перечислить все, если их несколько

In [111]:
jp_popular_publisher = vg_my_df.groupby('Publisher',as_index = False) \
    .agg({'JP_Sales' : 'mean'}) \
    .sort_values('JP_Sales', ascending = False)
jp_popular_publisher_top1 = jp_popular_publisher.iloc[0]['Publisher']

----

##### - Сколько игр продались лучше в Европе, чем в Японии?

In [127]:
e_b_j = vg_my_df.query('EU_Sales > JP_Sales').count()[0]

----

#####  - Загрузка 

In [ ]:
data = pd.read_csv(PATH)
vg_my_df = data.query('Year == @myYear')
return vg_my_df

----

#####  - Принт

In [ ]:
 def v_kachanovskij_24_print_data(max_sales_global, eu_mp_sales_top1, na_mp_platform_top1, jp_popular_publisher_top1, e_b_j):

        context = get_current_context()
        date = context['ds']

        print(f'''Game sales data in {year} for {date}:
        {max_sales_global} - best-selling game in the world;
            
        {eu_mp_sales_top1} - best-selling genre of games in Europe;
        
        {na_mp_platform_top1} - the most games that sold more than a million copies in North America;
            
        {jp_popular_publisher_top1} - the highest average sales in Japan;
            
        {e_b_j} games sold better in Europe than in Japan;''')
        